In [1]:
using Turing
using StatsPlots
using Distributions

In [9]:
@model function model(data)
    rt_max = mean(data) # Max RT
    return rt_max
    rt_max = 1000
    d_max = maximum(data.distance) # Max distance
    d_max = length(data)
    distance = [1:d_max]
    
    # Priors
    num_cps ~ Uniform(1,3) # Number of changepoints
    mu = DiscreteUniform.(rand(1:rt_max,num_cps+1)) # Means for each interval
    sigma ~ Normal(0,100)
    
    # The changepoints
    cps = [Uniform(0,d_max)] # Dealing with one changepoint
    if num_cps > 1 # Dealing with >1 changepoint
        for i in num_cps-1
            # Sample new Unif and add to vector
            push!(cps, rand(Uniform(cps[i], d_max))) 
        end
    end
    
    # Set up indexing variable for cps
    cp_idx = 1
    # Sample stationary intervals
    for d in 1:d_max
        if d < cps[cp_idx]
            data[d] ~ Normal(mu[cp_idx],sigma) # Make this a linear funciton of distance
        else
            cp_idx += 1
            data[d] ~ Normal(mu[cp_idx],sigma) # Make this a linear funciton of distance
        end
    end
end

model (generic function with 2 methods)

In [10]:
# generate data
num_cps_true = 2
cps = [100,500]
mu1 = 100
mu2 = 300
mu3 = 500
sigma = 20

x = vcat(
    rand(Normal(mu1,sigma), cps[1]), 
    rand(Normal(mu2,sigma), cps[2]-cps[1]),
    rand(Normal(mu3,sigma), 1000-cps[2]));

In [11]:
x

1000-element Vector{Float64}:
  62.9809412479054
  99.80685001925603
 103.94307356802149
  86.37687024608789
  72.99187164007742
 102.1573859717075
 122.73510787190139
  76.95684437296535
 109.31393372204853
  86.98994031146688
  98.62518405536875
 111.52239112148223
  89.5144701011275
   ⋮
 506.75645125273616
 507.60796312669834
 502.4795315347952
 535.8840781460851
 484.8778275298654
 476.98906765831657
 458.2724893738179
 476.05556624864715
 536.293318630134
 515.2238653854439
 486.0034519538988
 530.6809991530472

In [13]:
chain = sample(model(x), NUTS(), 5000)

plot(chain)

LoadError: MethodError: no method matching phasepoint(::Random._GLOBAL_RNG, ::Vector{Any}, ::AdvancedHMC.Hamiltonian{AdvancedHMC.DiagEuclideanMetric{Float64, Vector{Float64}}, Base.Fix1{typeof(LogDensityProblems.logdensity), LogDensityProblems.ForwardDiffLogDensity{Turing.LogDensityFunction{DynamicPPL.TypedVarInfo{NamedTuple{(), Tuple{}}, Float64}, DynamicPPL.Model{typeof(model), (:data,), (), (), Tuple{Vector{Float64}}, Tuple{}, DynamicPPL.DefaultContext}, DynamicPPL.Sampler{NUTS{Turing.Essential.ForwardDiffAD{0}, (), AdvancedHMC.DiagEuclideanMetric}}, DynamicPPL.DefaultContext}, ForwardDiff.GradientConfig{ForwardDiff.Tag{Turing.TuringTag, Any}, Any, 0, Vector{ForwardDiff.Dual{ForwardDiff.Tag{Turing.TuringTag, Any}, Any, 0}}}}}, Turing.Inference.var"#∂logπ∂θ#44"{LogDensityProblems.ForwardDiffLogDensity{Turing.LogDensityFunction{DynamicPPL.TypedVarInfo{NamedTuple{(), Tuple{}}, Float64}, DynamicPPL.Model{typeof(model), (:data,), (), (), Tuple{Vector{Float64}}, Tuple{}, DynamicPPL.Defaul